In [1]:
# import some packages
import pandas as pd
from collections import Counter
from tflearn.data_utils import pad_sequences
import random
import numpy as np
import h5py
import pickle
import codecs
print("import package successful...")

D:\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)
import package successful...


### 生成所需的文件

In [3]:
# 读取训练数据
base_path='D:/zhihu_data/data/ieee_zhihu_cup2/'
train_data_x=pd.read_csv(base_path+'question_train_set3.txt',sep='\t', encoding="utf-8")
train_data_y=pd.read_csv(base_path+'question_topic_train_set3.txt',sep='\t', encoding="utf-8")

train_data_x=train_data_x.fillna('')
train_data_y=train_data_y.fillna('')

print("train_data_x:",train_data_x.shape)
print("train_data_y:",train_data_y.shape)

FileNotFoundError: File b'/data/chenhy/data/ieee_zhihu_cup/question_train_set3.txt' does not exist

In [3]:
# 组成新的文件

#连接title和desc
train_data_x['title_desc_char'] = train_data_x.apply(lambda t: t['title_char'] + ',SEP,' + t['desc_char'], axis=1)
train_desc = pd.concat([train_data_x['title_desc_char'], train_data_y['topic_ids']],axis=1)

file = 'train-title-desc.txt' #c324,c39,c40,c155,SEP,c180,c180,c181,c17,c4,c5\t7739004195693774975,3738968195649774859
train_desc.to_csv(file, encoding='utf-8', sep='\t', index=None)

train_desc.head()

,title_desc_char,topic_ids
0,"c324,c39,c40,c155,c180,c180,c181,c17,c4,c1153,...","7739004195693774975,3738968195649774859"
1,"c44,c110,c101,c286,c106,c150,c101,c892,c632,c1...",-3149765934180654494
2,"c15,c768,c769,c1363,c650,c1218,c2361,c11,c90,c...",-760432988437306018
3,"c473,c1528,c528,c428,c295,c15,c101,c188,c146,c...","-6758942141122113907,3195914392210930723"
4,"c190,c147,c105,c219,c220,c101,c647,c219,c220,c...","3804601920633030746,4797226510592237555,435133..."


### 生成word2index和label2index，此处的word为char

In [4]:
#创建word2index表和embedding表
word_embedding_object=open(base_path+'unused_current/char_embedding.txt')
lines_wv=word_embedding_object.readlines()
word_embedding_object.close()
char_list=[]
char_list.extend(['PAD','UNK','SEP'])
PAD_ID=0
UNK_ID=1
EMBED_SIZE = 100
bound = 0.5
embedding = [None] * (int(lines_wv[0].split(' ')[0]) + 3) #长度
embedding[0] = np.zeros(100)
embedding[1] = np.random.rand(100)
embedding[2] = embedding[0] + 0.1
for i, line in enumerate(lines_wv):
    if i==0: continue
    char_embedding_list=line.split(" ")
    char_token=char_embedding_list[0]
    char_list.append(char_token)
    embedding[i+2] = char_embedding_list[1:1+EMBED_SIZE]

embedding = np.array(embedding,dtype=np.float32)
# write to vocab.txt under data/ieee_zhihu_cup2
vocab_path='vocab.txt'

vocab_char_object=open(vocab_path,'w')

word2index={}
for i, char in enumerate(char_list):
    if i<10:print(i,char)
    word2index[char]=i
    vocab_char_object.write(char+"\n")
vocab_char_object.close()
print("vocabulary of char generated....")

0 PAD
1 UNK
2 SEP
3 </s>
4 c17
5 c101
6 c11
7 c4
8 c147
9 c85
vocabulary of char generated....


In [5]:
 # generate labels list, and save to file system 
c_labels=Counter()
train_data_y_small=train_data_y[0:100000]#.sample(frac=0.1)
for index, row in train_data_y_small.iterrows():
    topic_ids=row['topic_ids']
    topic_list=topic_ids.split(',')
    c_labels.update(topic_list)

label_list=c_labels.most_common()
label2index={}
label_target_object=open('label_set.txt','w')
for i, label_freq in enumerate(label_list):
    label,freq=label_freq
    label2index[label]=i
    label_target_object.write(label+"\n")
    if i<20: print(label,freq)
label_target_object.close()
print("generate label dict successful...")

7476760589625268543 2308
4697014490911193675 1746
-4653836020042332281 1579
-8175048003539471998 1475
-8377411942628634656 1382
-7046289575185911002 1338
-5932391056759866388 1283
2787171473654490487 1145
-7129272008741138808 1085
2587540952280802350 1079
-4931965624608608932 1079
-6748914495015758455 1049
-5513826101327857645 993
2347973810368732059 970
9069451131871918127 958
-8132909213241034354 904
-3517637179126242000 867
-5872443091340192918 834
-3522198575349379632 830
1127459907694805235 829
generate label dict successful...


In [12]:
def load_data_multilabel_new(vocabulary_word2index,vocabulary_word2index_label,valid_portion=0.05,max_training_data=100,
                             traning_data_path='train-title-desc.txt',multi_label_flag=True,use_seq2seq=False,seq2seq_label_length=6):  # n_words=100000,
    """
    input: a file path
    :return: train, test, valid. where train=(trainX, trainY). where
                trainX: is a list of list.each list representation a sentence.trainY: is a list of label. each label is a number
    """
    # 1.load a zhihu data from file
    # example:"w305 w6651 w3974 w1005 w54 w109 w110 w3974 w29 w25 w1513 w3645 w6 w111 __label__-400525901828896492"
    print("load_data.started...")
    print("load_data_multilabel_new.training_data_path:",traning_data_path)
    zhihu_f = codecs.open(traning_data_path, 'r', 'utf8') #-zhihu4-only-title.txt
    lines = zhihu_f.readlines(101)
    # 2.transform X as indices
    # 3.transform  y as scalar
    X = []
    Y = []
    Y_decoder_input=[] #ADD 2017-06-15
    for i, line in enumerate(lines):
        x, y = line.split('\t') #x='w17314 w5521 w7729 w767 w10147 w111'
        y=y.strip().replace('\n','')
        x = x.strip()
        if i<1:
            print(i,"x0:",x) #get raw x
        #x_=process_one_sentence_to_get_ui_bi_tri_gram(x)
        x=x.split(",")
        x = [vocabulary_word2index.get(e,0) for e in x] #if can't find the word, set the index as '0'.(equal to PAD_ID = 0)
        if i<2:
            print(i,"x1:",x) #word to index
        if use_seq2seq:        # 1)prepare label for seq2seq format(ADD _GO,_END,_PAD for seq2seq)
            ys = y.replace('\n', '').split(",")  # ys is a list
            _PAD_INDEX=vocabulary_word2index_label[_PAD]
            ys_mulithot_list=[PAD_ID]*seq2seq_label_length #[3,2,11,14,1]
            ys_decoder_input=[PAD_ID]*seq2seq_label_length
            # below is label.
            for j,y in enumerate(ys):
                if j<seq2seq_label_length-1:
                    ys_mulithot_list[j]=vocabulary_word2index_label[y]
            if len(ys)>seq2seq_label_length-1:
                ys_mulithot_list[seq2seq_label_length-1]=vocabulary_word2index_label[_END]#ADD END TOKEN
            else:
                ys_mulithot_list[len(ys)] = vocabulary_word2index_label[_END]

            # below is input for decoder.
            ys_decoder_input[0]=vocabulary_word2index_label[_GO]
            for j,y in enumerate(ys):
                if j < seq2seq_label_length - 1:
                    ys_decoder_input[j+1]=vocabulary_word2index_label[y]
            if i<10:
                print(i,"ys:==========>0", ys)
                print(i,"ys_mulithot_list:==============>1", ys_mulithot_list)
                print(i,"ys_decoder_input:==============>2", ys_decoder_input)
        else:
            if multi_label_flag: # 2)prepare multi-label format for classification
                ys = y.replace('\n', '').split(",")  # ys is a list
                ys_index=[]
                for y in ys:
                    y_index = vocabulary_word2index_label[y]
                    ys_index.append(y_index)
                ys_mulithot_list=transform_multilabel_as_multihot(ys_index)
            else:                #3)prepare single label format for classification
                ys_mulithot_list=vocabulary_word2index_label[y]
        if i<=3:
            print("ys_index:")
            #print(ys_index)
            print(i,"y:",y," ;ys_mulithot_list:",ys_mulithot_list) #," ;ys_decoder_input:",ys_decoder_input)
        X.append(x)
        Y.append(ys_mulithot_list)
        if use_seq2seq:
            Y_decoder_input.append(ys_decoder_input) #decoder input
        #if i>50000:
        #    break
    # 4.split to train,test and valid data
    number_examples = len(X)
    print("number_examples:",number_examples) #
    train = (X[0:int((1 - valid_portion) * number_examples)], Y[0:int((1 - valid_portion) * number_examples)])
    test = (X[int((1 - valid_portion) * number_examples) + 1:], Y[int((1 - valid_portion) * number_examples) + 1:])
    if use_seq2seq:
        train=train+(Y_decoder_input[0:int((1 - valid_portion) * number_examples)],)
        test=test+(Y_decoder_input[int((1 - valid_portion) * number_examples) + 1:],)
    # 5.return
    print("load_data.ended...")
    return train, test, test

In [13]:
load_data_multilabel_new(word2index, label2index, use_seq2seq=True)

load_data.started...
load_data_multilabel_new.training_data_path: train-title-desc.txt


MemoryError: 